![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Clinical Entity Resolvers

In [3]:
import os

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

2.4.2

In [4]:
# if you want to load the licensed models from S3 with your license key

! pip install awscli --upgrade

In [5]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.247.156:7077 
 AppName 
 Databricks Shell

# Clinical Resolvers

## Entity Resolvers for ICD-10

In [8]:
import pandas as pd
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel
import pyspark.sql.functions as F
import string
import numpy as np
import sparknlp
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F
from sparknlp_jsl.annotator import *

In [9]:
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [10]:
!aws s3 cp s3://auxdata.johnsnowlabs.com/clinical/models/chunkresolve_icd10cm_clinical_en_2.4.2_2.4_1583085234727.zip chunkresolve_icd10cm_clinical_en_2.4.2_2.4_1583085234727.zip

In [11]:
!unzip chunkresolve_icd10cm_clinical_en_2.4.2_2.4_1583085234727.zip -d chunkresolve_icd10cm

In [12]:
%fs ls "file:/databricks/driver"

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/derby.log,derby.log,726
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/i2b2_assertion_sample.csv,i2b2_assertion_sample.csv,141585
file:/databricks/driver/chunkresolve_icd10cm_clinical_en_2.4.2_2.4_1583085234727.zip,chunkresolve_icd10cm_clinical_en_2.4.2_2.4_1583085234727.zip,105024355
file:/databricks/driver/chunkresolve_icd10cm/,chunkresolve_icd10cm/,4096
file:/databricks/driver/logs/,logs/,4096


In [13]:
dbutils.fs.cp("file:/databricks/driver/chunkresolve_icd10cm", "dbfs:/chunkresolve_icd10cm", recurse=True)

Out[12]: True

In [14]:
%fs ls "dbfs:/"

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/GMB_NER.conll,GMB_NER.conll,1663772
dbfs:/andres/,andres/,0
dbfs:/chunkresolve_icd10cm/,chunkresolve_icd10cm/,0
dbfs:/cluster-logs/,cluster-logs/,0
dbfs:/context-spell-checker/,context-spell-checker/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/datasets/,datasets/,0


In [15]:
icd_ner_converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_chunk")\
.setWhiteList(['PROBLEM'])

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("ner_chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")\

chunk_tokenizer = ChunkTokenizer()\
    .setInputCols("ner_chunk")\
    .setOutputCol("ner_token")


entity_resolver_icd10 = ChunkEntityResolverModel.load("dbfs:/chunkresolve_icd10cm")\
         .setInputCols(["ner_token", "chunk_embeddings"])\
         .setOutputCol("icd10cm_code")\
         .setDistanceFunction("COSINE")

pipeline_icd10 = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    icd_ner_converter,
    chunk_embeddings,
    chunk_tokenizer,
    entity_resolver_icd10
  ])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model_icd10 = pipeline_icd10.fit(empty_data)


In [16]:
%sh
wget 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv


In [17]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("pubmed_sample_text_small.csv").repartition(4)
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
Studies of the past several decades have provid...|
Neurons in the rostroventrolateral medulla (RVL...|
In a model of psychological dependence, a very ...|
To determine the functional alterations of the ...|
"""A new scheme has been described for continuo...|
OBJECTIVE: Although present treatment programs ...|
PURPOSE: To compare image quality and lesion de...|
The identification of gluten peptides eliciting...|
In two experiments, undergraduates processed a ...|
The author analyses certain aspects of the narr...|
BACKGROUND: The associations between diet and c...|
Properdin type I deficiency is characterised by...|
BACKGROUND: A transjugular intrahepatic portosy...|
AIM: To study the activation of cytotoxic T lym...|
Type II transmembrane serine proteases are an e...|
This paper describes a Japanese logographic cha...|
Immunoscintigraphy with 111In-labeled anti-CEA-...|
Arene ligand exchange in the (eta(6)-arene)Cr(C...|
The whole process of EAFP protein monoclinic cr...|
In this case report we illustrate our experienc...|
+--------------------------------------------------+
only showing top 20 rows

In [18]:
result = model_icd10.transform(pubMedDF.limit(100))

In [19]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| chunk_embeddings| ner_token| icd10cm_code|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Studies of the pa...|[[document, 0, 84...|[[document, 0, 17...|[[token, 0, 6, St...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 440, 449...|[[word_embeddings...|[[token, 440, 444...|[[entity, 440, 44...|
Neurons in the ro...|[[document, 0, 15...|[[document, 0, 12...|[[token, 0, 6, Ne...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 585, 642...|[[word_embeddings...|[[token, 585, 587...|[[entity, 585, 64...|
In a model of psy...|[[document, 0, 10...|[[document, 0, 23...|[[token, 0, 1, In...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 14, 37, ...|[[word_embeddings...|[[token, 14, 26, ...|[[entity, 14, 37,...|
To determine the ...|[[document, 0, 10...|[[document, 0, 36...|[[token, 0, 1, To...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 81, ...|[[word_embeddings...|[[token, 13, 15, ...|[[entity, 13, 81,...|
"""A new scheme h...|[[document, 0, 16...|[[document, 0, 10...|[[token, 0, 2, ""...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 145, 162...|[[word_embeddings...|[[token, 145, 162...|[[entity, 0, 0, ,...|
OBJECTIVE: Althou...|[[document, 0, 15...|[[document, 0, 20...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 51, 70, ...|[[word_embeddings...|[[token, 51, 55, ...|[[entity, 51, 70,...|
PURPOSE: To compa...|[[document, 0, 15...|[[document, 0, 31...|[[token, 0, 6, PU...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 38, 43, ...|[[word_embeddings...|[[token, 38, 43, ...|[[entity, 38, 43,...|
The identificatio...|[[document, 0, 17...|[[document, 0, 15...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 22, 36, ...|[[word_embeddings...|[[token, 22, 27, ...|[[entity, 22, 36,...|
In two experiment...|[[document, 0, 19...|[[document, 0, 14...|[[token, 0, 1, In...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 69, 90, ...|[[word_embeddings...|[[token, 69, 69, ...|[[entity, 69, 90,...|
The author analys...|[[document, 0, 13...|[[document, 0, 29...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 56, 103,...|[[word_embeddings...|[[token, 56, 56, ...|[[entity, 56, 103...|
BACKGROUND: The a...|[[document, 0, 69...|[[document, 0, 13...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 46, 51, ...|[[word_embeddings...|[[token, 46, 51, ...|[[entity, 46, 51,...|
Properdin type I ...|[[document, 0, 10...|[[document, 0, 16...|[[token, 0, 8, Pr...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 26, P...|[[word_embeddings...|[[token, 0, 8, Pr...|[[entity, 0, 26, ...|
BACKGROUND: A tra...|[[document, 0, 16...|[[document, 0, 17...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 124, 169...|[[word_embeddings...|[[token, 124, 133...|[[entity, 124, 16...|
AIM: To study the...|[[document, 0, 16...|[[document, 0, 26...|[[token, 0, 2, AI...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 212, 222...|[[word_embeddings...|[[token, 212, 216...|[[entity, 212, 22...|
Type II transmemb...|[[document, 0, 16...|[[document, 0, 15...|[[token, 0, 3, Ty...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 37, T...|[[word_embeddings...|[[token, 0, 3, Ty...|[[entity, 0, 37, ...|
This paper descri...|[[document, 0, 74...|[[document, 0, 52...|[[token, 0, 3, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 390, 419...|[[word_embeddings...|[[token, 390, 393...|[[entity, 390, 41...|
Immunoscintigraph...|[[document, 0, 99...|[[document, 0, 20...|[[token, 0, 17, I...|[[word_embe

In [20]:
def get_icd10_codes(model, text):
    
    data = spark.createDataFrame([[text]]).toDF("text")
        
    output = model.transform(data)
    
    result = output.select(F.explode(F.arrays_zip("ner_chunk.begin",
                                         "ner_chunk.end",
                                         "ner_chunk.result", "ner_chunk.metadata", "icd10cm_code.result", "icd10cm_code.metadata")).alias("icd10cm_result")) \
    .select(F.expr("icd10cm_result['2']").alias("ner"),
            F.expr("icd10cm_result['3'].entity").alias("entity"),
            F.expr("icd10cm_result['0']").alias("begin"),
            F.expr("icd10cm_result['1']").alias("end"),
            F.expr("icd10cm_result['4']").alias("code"),
            F.expr("icd10cm_result['5'].resolved_text").alias("resolved_text"),
            F.expr("icd10cm_result['5'].alternative_codes").alias("alternative_codes"),
            F.expr("round(icd10cm_result['5'].confidence_ratio,2)").alias("confidence_ratio"))\
    .distinct().toPandas()

    return result

In [21]:
text = 'He has a brain damage and lung cancer'

df = get_icd10_codes(model_icd10, text)

df

,ner,entity,begin,end,code,resolved_text,alternative_codes,confidence_ratio
0,lung cancer,PROBLEM,26,36,C220,Liver cell carcinoma,C61:|:C61:|:J675:|:J674:|:J670:|:J672:|:J670:|...,1.38
1,a brain damage,PROBLEM,7,20,Z87820,Personal history of traumatic brain injury,P112:|:Z13850:|:P112:|:G931:|:P111:|:S062X7A:|...,1.02


# RxNorm Resolver

In [23]:
rxnorm_resolution_l1 = DocumentLogRegClassifierModel.pretrained("resolve_rxnorm_clinical_l1", "en", "clinical/models")\
    .setInputCols("ner_token").setOutputCol("partition")

rxnorm_resolution_l2 = ResourceDownloader.downloadPipeline("resolve_rxnorm_clinical_l2", "en", "clinical/models")

rxnorm_resolution = PipelineModel([rxnorm_resolution_l1, RecursivePipelineModel(rxnorm_resolution_l2)])

posology_ner_model_large = NerDLModel.pretrained('ner_posology', "en", "clinical/models")\
            .setInputCols(["sentence", "token", "embeddings"]) \
          .setOutputCol("ner")

drug_converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_chunk")\
.setWhiteList(['DRUG'])

pipeline_rx = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner_model_large,
    drug_converter,
    chunk_embeddings,
    chunk_tokenizer,
    rxnorm_resolution
  ])

model_rxnorm = pipeline_rx.fit(empty_data)


resolve_rxnorm_clinical_l1 download started this may take some time.
Approximate size to download 7.5 MB
[ | ][OK!]
resolve_rxnorm_clinical_l2 download started this may take some time.
Approx size to download 276.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][OK!]

In [24]:

def get_rxnorm_codes(text):
    
    data = spark.createDataFrame([[text]]).toDF("text")
        
    output = model_rxnorm.transform(data)
    
    result = output.select(F.explode(F.arrays_zip("ner_chunk.begin",
                                     "ner_chunk.end",
                                     "ner_chunk.result", "ner_chunk.metadata", "rxnorm_code.result", "rxnorm_code.metadata")).alias("rxnorm_result")) \
    .select(F.expr("rxnorm_result['2']").alias("ner"),
            F.expr("rxnorm_result['3'].entity").alias("entity"),
            F.expr("rxnorm_result['0']").alias("begin"),
            F.expr("rxnorm_result['1']").alias("end"),
            F.expr("rxnorm_result['4']").alias("code"),
            F.expr("rxnorm_result['5'].resolved_text").alias("resolved_text"),
            F.expr("rxnorm_result['5'].alternative_codes").alias("alternative_codes"),
            F.expr("round(rxnorm_result['5'].confidence_ratio,2)").alias("confidence_ratio")) \
    .distinct()\
    .toPandas()
    
    return result



In [25]:

text = 'He has a brain damage and needs to take an Advil and Aspirin'

get_rxnorm_codes(text)

,ner,entity,begin,end,code,resolved_text,alternative_codes,confidence_ratio
0,Advil,DRUG,43,47,352893,phoslo gelcap,1941952:|:1318187:|:827207:|:19711,1.01
1,Aspirin,DRUG,53,59,238143,aspirin pwdr,238143:|:687078:|:1986390:|:226716,1.00


## Snomed Resolver

In [27]:
 #SNOMED Resolution
ner_snomed_resolver_l1 = DocumentLogRegClassifierModel.pretrained("resolve_snomed_clinical_l1", "en", "clinical/models")\
    .setInputCols("ner_token").setOutputCol("partition")

ner_snomed_resolver_l2 = ResourceDownloader.downloadPipeline("resolve_snomed_clinical_l2", "en", "clinical/models")


ner_snomed_resolver_l2.stages[-1].setInputCols("partition","ner_token","chunk_embeddings")


resolve_snomed_clinical_l1 download started this may take some time.
Approximate size to download 15.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
resolve_snomed_clinical_l2 download started this may take some time.
Approx size to download 583.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
Out[48]: ChunkEntityResolverSelector_d41a7a88595b

In [28]:
from pyspark.ml import PipelineModel

snomed_resolution = PipelineModel([ner_snomed_resolver_l1, RecursivePipelineModel(ner_snomed_resolver_l2)])


In [29]:
clinical_converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_chunk")


def get_snomed_model():
    pipeline_snomed = Pipeline(
        stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        posology_ner_model_large,
        clinical_converter,
        chunk_embeddings,
        chunk_tokenizer,
        snomed_resolution
      ])

    model_snomed = pipeline_snomed.fit(empty_data)
    
    return model_snomed


model_snomed = get_snomed_model()

In [30]:
def get_snomed_codes(text):
    
    data = spark.createDataFrame([[text]]).toDF("text")
    
    output = model_snomed.transform(data)
        
    result = output.select(F.explode(F.arrays_zip("ner_chunk.begin",
                                     "ner_chunk.end",
                                         "ner_chunk.result", "ner_chunk.metadata", "snomed_code.result", "snomed_code.metadata")).alias("snomed_result")) \
    .select(F.expr("snomed_result['2']").alias("ner"),
            F.expr("snomed_result['3'].entity").alias("entity"),
            F.expr("snomed_result['0']").alias("begin"),
            F.expr("snomed_result['1']").alias("end"),
            F.expr("snomed_result['4']").alias("code"),
            F.expr("snomed_result['5'].resolved_text").alias("resolved_text"),
            F.expr("snomed_result['5'].alternative_codes").alias("alternative_codes"),
            F.expr("round(snomed_result['5'].confidence_ratio,2)").alias("confidence_ratio")) \
    .distinct()\
    .toPandas()

    return result


In [31]:
text = 'He has a brain damage and needs to take Parol or Aspirin'

df = get_snomed_codes(text)

df

,ner,entity,begin,end,code,resolved_text,alternative_codes,confidence_ratio
0,Aspirin,DRUG,49,55,312452009,aspirin prophylaxis,717854002:|:413081008:|:243195000:|:243195000,1.05
1,Parol,DRUG,40,44,,,,NaN
